In [2]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [3]:
#### GET react, angular, vue.js QUESTIONS ##############

sample_size = 20000

query = '''
    SELECT RAND() as r, id, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE (REGEXP_CONTAINS(tags, r"vue.js") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular"))
    ORDER BY r
    LIMIT ''' + str(sample_size)

df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df

,r,id,tags,creation_date
0,9.467786e-07,53763078,twitter-bootstrap|typescript|angular2-template...,2018-12-13 13:34:33.387000+00:00
1,1.153978e-06,28349546,javascript|css|angularjs,2015-02-05 16:49:21.420000+00:00
2,2.698203e-06,35769951,html|angularjs|selenium,2016-03-03 10:45:13.417000+00:00
3,3.467710e-06,27061890,angularjs|angularjs-scope|angularjs-injector,2014-11-21 12:53:33.067000+00:00
4,3.485237e-06,39903325,javascript|angularjs|angular-components|angula...,2016-10-06 18:33:56.357000+00:00
...,...,...,...,...
19995,2.186995e-02,50472899,javascript|angular|angular2-template,2018-05-22 16:55:20.140000+00:00
19996,2.187005e-02,47476812,angular|cors|.net-core|adfs4.0,2017-11-24 16:09:47.427000+00:00
19997,2.187075e-02,38289989,ruby-on-rails|angularjs|ionic-framework,2016-07-10 08:18:32.437000+00:00
19998,2.187211e-02,46452347,javascript|angular|typescript|rxjs,2017-09-27 15:58:32.067000+00:00


In [13]:
##### CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(x.find('react') != -1):
        return 'react'
    elif(x.find('angular') != -1):
        return 'angular'
    elif(x.find('vue.js') != -1):
        return 'vue.js'
    

toSaveDf = pd.DataFrame(df)
toSaveDf = toSaveDf.drop(columns=['id', 'r'])
toSaveDf.rename(columns={'tags':'framework'}, inplace=True)
toSaveDf['framework'] = toSaveDf['framework'].map(setTag)
toSaveDf.to_csv('js-framework-tags-dataframe')

In [ ]:
tagDf = myDf.drop(columns='r')
tagDf['count'] = 1
tagDf = tagDf.groupby(['tags', 'creation_date']).count().unstack()

print(tagDf['count'])
hist = px.histogram(tagDf, x=['view.js', 'angular', 'react'], color=[2017,2018,2019], y='count', title="JS Framework Interests in 2019 (based on 5k sample)", barmode="group")
hist.show()